# Иерархия памяти CPU

Попытаемся ответить на вопрос, что происходит в железе, когда мы выполняем такой код, сколько тактов он работает:

```c++
int b = 5;

...;

int a = b + 3;  // b - переменная на стеке (предположим, компилятор во время оптимизаций её не выкинул)
```

_(Рисовать как дотянуть переменную `b` из RAM до регистров и зачем нужны кеши)_

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />


![](mem_hierarchy_single_cpu.png)

<br />

**Cache line**:

Когда данные идут из памяти в регистры через кеши, они идут блоками по (обычно) 64 байта. Этими же блоками они оседают в кешах памяти. Такой блок называется cache line.

`64 byte == 8 x int64_t == 8 x double == 16 x int32_t == 16 x float`

Рассмотрим функцию, суммирующую элементы массива:

```c++
float sum(const std::vector<float>& v)
{
    float rv = 0;
    for (float x : v)
        rv += x;
    return rv;
}
```

_(Рисовать как будут бегать данные между памятью и кешами для этой функции)_

<br />

**За пределами рассказа**:
* ассоциативность кешей
* многопоточность и cache line, разреженность
* NUMA